In [63]:
import os
import json
import re
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

In [64]:
all_data_dir = '../../results-demo/public/data' 

count_folder_names = [os.path.join(all_data_dir, 'count-only-' + str(x)) for x in range(1,5)]
print(count_folder_names)

as_folder_names = [os.path.join(all_data_dir, 'final-' + str(x)) for x in range(1,5)]
print(as_folder_names)

# ee_folder_names = [os.path.join(all_data_dir, 'param-ee-quadratic-' + x) for x in ['0.01', '0.07', '0.13', '0.20', '0.26', '0.328']]
# print(ee_folder_names)

# nn_folder_names = [os.path.join(all_data_dir, 'final-with-dunne-metrics')]
# ee_folder_names = [os.path.join(all_data_dir, 'final-with-dunne-metrics')]

['../../results-demo/public/data/count-only-1', '../../results-demo/public/data/count-only-2', '../../results-demo/public/data/count-only-3', '../../results-demo/public/data/count-only-4']
['../../results-demo/public/data/final-1', '../../results-demo/public/data/final-2', '../../results-demo/public/data/final-3', '../../results-demo/public/data/final-4']


In [65]:
graphs_file_path = os.path.join(as_folder_names[0], 'graphs.txt')
graph_names = [l.strip() for l in open(graphs_file_path)]
print(graph_names)
print(len(graph_names))

['four-clusters-ne0', 'four-clusters-ne1', 'four-clusters-nn0', 'four-clusters-nn1', 'four-clusters-nn2', 'four-clusters-nn4', 'four-clusters-sprawl0', 'four-clusters-sprawl1', 'four-clusters-ee0', 'four-clusters-ee-glancing', 'special-cases-1', 'special-cases-2', 'special-cases-3', 'special-cases-4', 'special-cases-5', 'special-cases-6', 'special-cases-7', 'progression-nn-none', 'progression-nn-touch-leaf', 'progression-nn-touch-meta', 'progression-nn-some-leaf', 'progression-nn-some-meta', 'progression-nn-near-max-leaf', 'progression-nn-near-max-meta', 'progression-ne-touch-leaf', 'progression-ne-touch-meta', 'progression-ne-some-leaf', 'progression-ne-some-meta', 'progression-ne-near-max-leaf', 'progression-ne-near-max-meta', 'progression-ee-ortho', 'progression-ee-half', 'progression-ee-near-glancing', 'progression-ee-glancing', 'coauthor-main-comp-grouseflocks-0', 'coauthor-main-comp-grouseflocks-1', 'coauthor-main-comp-grouseflocks-2', 'coauthor-main-comp-gem', 'coauthor-main-com

In [66]:
def comp_avr(g, folders, family):
    s = 0
    for f in folders:
        res_path = os.path.join(f, g + '_result.json')
        data = json.load(open(res_path))
        s += data['metrics'][family]['execution_time']
    return s / len(folders)

In [67]:
# Use shorter name in the table since we ran out of horizontal space in supplemental material
def layout_name_mapping(g):
    g2 = g.replace('-main-comp-', '-')
    g3 = g2.replace('-open-', '-')
    g4 = g3.replace('grouseflocks-ivOrigins', 'ivOrigins')
    g5 = g4.replace('email-eu-core', 'email')
    return g5

In [68]:
df_arr = []

# Extract the computational times and compute the average
for g in graph_names:
    df_arr.append({
        'layout': layout_name_mapping(g),
        'count-nn': comp_avr(g, count_folder_names, 'nn'),
        'count-ne': comp_avr(g, count_folder_names, 'ne'),
        'count-ee': comp_avr(g, count_folder_names, 'ee'),
        'as-nn': comp_avr(g, as_folder_names, 'nn'),
        'as-ne': comp_avr(g, as_folder_names, 'ne'),
        'as-ee': comp_avr(g, as_folder_names, 'ee')
    })

In [69]:
d = pd.DataFrame(df_arr, columns=['layout', 'as-nn', 'count-nn', 'as-ne', 'count-ne', 'as-ee', 'count-ee'])

In [70]:
d['ratio-nn'] = np.where(d['count-nn'] > 1, d['as-nn'] / d['count-nn'], np.NaN)
d['ratio-ne'] = np.where(d['count-ne'] > 1, d['as-ne'] / d['count-ne'], np.NaN)
d['ratio-ee'] = np.where(d['count-ee'] > 1, d['as-ee'] / d['count-ee'], np.NaN)

In [71]:
d

,layout,as-nn,count-nn,as-ne,count-ne,as-ee,count-ee,ratio-nn,ratio-ne,ratio-ee
0,four-clusters-ne0,0.012024,0.012159,0.089342,0.089088,0.125637,0.116664,NaN,NaN,NaN
1,four-clusters-ne1,0.013058,0.011846,0.108848,0.101580,0.175018,0.150606,NaN,NaN,NaN
2,four-clusters-nn0,0.013561,0.012460,0.090650,0.087854,0.127350,0.117972,NaN,NaN,NaN
3,four-clusters-nn1,0.012859,0.013507,0.104260,0.098660,0.131417,0.123631,NaN,NaN,NaN
4,four-clusters-nn2,0.017364,0.012653,0.124948,0.087511,0.143251,0.108884,NaN,NaN,NaN
5,four-clusters-nn4,0.014502,0.012269,0.112854,0.069277,0.128543,0.101806,NaN,NaN,NaN
6,four-clusters-sprawl0,0.012932,0.010663,0.103211,0.073005,0.156169,0.118315,NaN,NaN,NaN
7,four-clusters-sprawl1,0.011858,0.010150,0.076812,0.070705,0.124064,0.106175,NaN,NaN,NaN
8,four-clusters-ee0,0.010290,0.009763,0.096423,0.077503,0.169154,0.120621,NaN,NaN,NaN
9,four-clusters-ee-glancing,0.010895,0.009033,0.103806,0.065454,0.160709,0.110166,NaN,NaN,NaN


In [72]:
d[d['count-nn'] > 1.0]['ratio-nn'].describe()

count    6.000000
mean     1.443293
std      0.156744
min      1.280896
25%      1.319976
50%      1.413876
75%      1.541135
max      1.678880
Name: ratio-nn, dtype: float64

In [73]:
d[d['count-ne'] > 1]['ratio-ne'].describe()

count    12.000000
mean      1.487126
std       0.236916
min       1.141520
25%       1.348270
50%       1.465123
75%       1.595681
max       2.052965
Name: ratio-ne, dtype: float64

In [74]:
d[d['count-ee'] > 1.0]['ratio-ee'].describe()

count    14.000000
mean      1.985477
std       0.474243
min       1.419247
25%       1.680493
50%       1.752845
75%       2.227940
max       2.918722
Name: ratio-ee, dtype: float64

In [75]:
# Generate a latex table

none_formatter = lambda x: str(x)
rounding = lambda x: 'NA' if np.isnan(x) else '{:.2f}'.format(x)

latex_str = d.to_latex(None, index=False, column_format='l|rrr|rrr|rrr', 
           columns=['layout', 'as-nn', 'count-nn', 'ratio-nn', 'as-ne', 'count-ne', 'ratio-ne', 
                    'as-ee', 'count-ee', 'ratio-ee'],
           formatters=[none_formatter] + [rounding for _ in range(9)],
           header=['Layout', 'AS', 'count', 'slowdown', 'AS', 'count', 'slowdown', 'AS', 'count', 'ratio'])

In [76]:
avr_nn = d[d['count-nn'] > 1.0]['ratio-nn'].mean()
avr_ne = d[d['count-ne'] > 1.0]['ratio-ne'].mean()
avr_ee = d[d['count-ee'] > 1.0]['ratio-ee'].mean()

multicolumn_line = r'& \multicolumn{3}{c|}{NN} & \multicolumn{3}{c|}{NE} & \multicolumn{3}{c}{EE} \\'
midrule_line = r'\midrule'
avr_line = r'\textbf{average} ' + '& & & {:.2f} & & & {:.2f} & & & {:.2f} '.format(avr_nn, avr_ne, avr_ee) + r'\\'

lines = latex_str.split('\n')
lines.insert(2, multicolumn_line)
lines.insert(len(lines) - 3, midrule_line)
lines.insert(len(lines) - 3, avr_line)

In [77]:
f = open('running_times_table.tex', 'w')
f.write('\n'.join(lines))
f.close()